In [2]:
import pandas as pd 

In [7]:
data_dir_path = '../data/'


In [25]:
def get_nyc_zcta5():
    zcta5_file_name = 'zip_to_zcta10_nyc_revised.xls'
    zcta5_df = pd.read_excel(data_dir_path + zcta5_file_name)
    zcta5_list = [str(x) for x in set(zcta5_df['zcta5'])]
    return zcta5_list
zcta5_list = get_nyc_zcta5()
print('length of zcta5_list: ', len(zcta5_list))
print('zcta5_list[:10]: ', zcta5_list[:10])

length of zcta5_list:  214
zcta5_list[:10]:  ['10271', '10278', '10279', '10280', '10282', '10301', '10302', '10303', '10304', '10305']


In [46]:
def get_nyc_only_df(input_df, zcta5_list):
    nyc_geo_list = ['8600000US' + x for x in zcta5_list]
    print('nyc_geo_list[:10]: ', nyc_geo_list[:10])
    # print(input_df['GEO_ID'].isin(nyc_geo_list))
    nyc_only_df = input_df[input_df['GEO_ID'].isin(nyc_geo_list)]
    nyc_only_df.index = range(len(nyc_only_df)) # set the index from 0
    print()
    print('nyc_only_df rows: ', nyc_only_df.shape[0])
    return nyc_only_df

### check the population: B01001
the total population of these 214 zip code ares is 8551905

In [76]:
# read data
B01001_dir_path = 'ACSDT5Y2018.B01001_2020-04-15T120015/'
B01001_file_name = 'ACSDT5Y2018.B01001_data_with_overlays_2020-04-15T120001.csv'
B01001_df = pd.read_csv(data_dir_path + B01001_dir_path + B01001_file_name)

# get nyc only data frame
B01001_nyc_only_df = get_nyc_only_df(B01001_df, zcta5_list)

# get the needed features(columns)
temp_total_population_num_feature = 'B01001_001E' # Estimate!!Total

# initialization
population_dict = {}

# result data frame
for i, row in B01001_nyc_only_df.iterrows():
    # we need to record the population of different zip code areas,
    # Since this is the first time to encounter population in different zip code areas.
    if row['GEO_ID'] in population_dict:
        print('we already have one')
    population_dict[row['GEO_ID']] = row[temp_total_population_num_feature]
    
# test if we get the right population
# print('population_dict: ', population_dict)
print('sum of population dict value: ', sum([v for _, v in population_dict.items()]))


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
sum of population dict value:  8551905


/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1. Ratio of women to men (all ages): B05003

In [110]:
# read data
B05003_dir_path = 'ACSDT5Y2018.B05003_2020-04-15T173748/'
B05003_file_name = 'ACSDT5Y2018.B05003_data_with_overlays_2020-04-15T173740.csv'
B05003_df = pd.read_csv(data_dir_path + B05003_dir_path + B05003_file_name)

# get nyc only data frame
B05003_nyc_only_df = get_nyc_only_df(B05003_df, zcta5_list)

# get the needed features(columns)
male_num_feature = 'B05003_002E' # Estimate!!Total!!Male
female_num_feature = 'B05003_013E' # Estimate!!Total!!Female

# result data frame
result_df = pd.DataFrame(columns=['GEO_ID', 'NAME', 'Ratio_women_to_men'])
count_exception = 0
for i, row in B05003_nyc_only_df.iterrows():
    try:
        result_df.loc[i] = [row['GEO_ID'], row['NAME'], row[female_num_feature]/row[male_num_feature]]
    except:
        # print(i)
        # print('row[female_num_feature]: ', row[female_num_feature])
        # print('row[male_num_feature]: ', row[male_num_feature])
        count_exception += 1
        result_df.loc[i] = [row['GEO_ID'], row['NAME'], 'None'] # Both row[female_num_feature] and row[male_num_feature] is 0

print('count_exception: ', count_exception)
result_df.head()
        

/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men
0,8600000US11222,ZCTA5 11222,1.02733
1,8600000US11233,ZCTA5 11233,1.18882
2,8600000US11235,ZCTA5 11235,1.0872
3,8600000US11426,ZCTA5 11426,1.00241
4,8600000US11427,ZCTA5 11427,1.0223


### 2. Percent Hispanic or Latino of any race: B03002

In [111]:
# read data
B03002_dir_path = 'ACSDT5Y2018.B03002_2020-04-15T173950/'
B03002_file_name = 'ACSDT5Y2018.B03002_data_with_overlays_2020-04-15T173937.csv'
B03002_df = pd.read_csv(data_dir_path + B03002_dir_path + B03002_file_name)

# get nyc only data frame
B03002_nyc_only_df = get_nyc_only_df(B03002_df, zcta5_list)

# get the needed features(columns)
population_num_feature = 'B03002_001E' # Estimate!!Total
hispanic_or_latino_num_feature = 'B03002_012E' # Estimate!!Total!!Hispanic or Latino

# double check the population: initialization
population_dict_2 = {}

# result data frame
result_df['Percent_hispanic_or_latino'] = 'None'
count_exception = 0
for i, row in B03002_nyc_only_df.iterrows():
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_hispanic_or_latino'] = row[hispanic_or_latino_num_feature]/row[population_num_feature]
    except:
        # print(i)
        # print('row[hispanic_or_latino_num_feature]: ', row[hispanic_or_latino_num_feature])
        # print('row[total_ethnicity_num_feature]: ', row[total_ethnicity_num_feature])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_hispanic_or_latino'] = 'None' # Both value are zeros
    
    # double check the population
    if row['GEO_ID'] in population_dict_2:
        print('we already have one')
    population_dict_2[row['GEO_ID']] = row[population_num_feature]

print('count_exception: ', count_exception)

# test if we get the right population
# print('population_dict_2: ', population_dict_2)
print('sum of population dict value: ', sum([v for _, v in population_dict_2.items()])) # 8551905 is as same as before
    
result_df.head()


/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29
sum of population dict value:  8551905


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino
0,8600000US11222,ZCTA5 11222,1.02733,0.142415
1,8600000US11233,ZCTA5 11233,1.18882,0.159804
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816
3,8600000US11426,ZCTA5 11426,1.00241,0.189318
4,8600000US11427,ZCTA5 11427,1.0223,0.194533


### 3. Percent white (alone or in combination): B02008

In [112]:
# read data
B02008_dir_path = 'ACSDT5Y2018.B02008_2020-04-15T181228/'
B02008_file_name = 'ACSDT5Y2018.B02008_data_with_overlays_2020-04-15T181224.csv'
B02008_df = pd.read_csv(data_dir_path + B02008_dir_path + B02008_file_name)

# get nyc only data frame
B02008_nyc_only_df = get_nyc_only_df(B02008_df, zcta5_list)
B02008_nyc_only_df

# get the needed features(columns)
white_num_feature = 'B02008_001E' # Estimate!!Total

# print('population_dict: ', population_dict)
# # result data frame
result_df['Percent_white'] = 'None'
count_exception = 0
for i, row in B02008_nyc_only_df.iterrows():   
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_white'] = float(row[white_num_feature])/population_dict[row['GEO_ID']]
    except:
        # print(i)
        # print('row[white_num_feature]: ', row[white_num_feature])
        # print('population_dict[row[GEO_ID]]: ', population_dict[row['GEO_ID']])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_white'] = 'None' # Both value are zeros

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646


### 4. Percent black or African American (alone or in combination): B02009

In [113]:
# read data
B02009_dir_path = 'ACSDT5Y2018.B02009_2020-04-15T180039/'
B02009_file_name = 'ACSDT5Y2018.B02009_data_with_overlays_2020-04-15T180030.csv'
B02009_df = pd.read_csv(data_dir_path + B02009_dir_path + B02009_file_name)

# get nyc only data frame
B02009_nyc_only_df = get_nyc_only_df(B02009_df, zcta5_list)
B02009_nyc_only_df

# get the needed features(columns)
black_num_feature = 'B02009_001E' # Estimate!!Total

# # result data frame
result_df['Percent_black'] = 'None'
count_exception = 0
for i, row in B02009_nyc_only_df.iterrows():   
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_black'] = float(row[black_num_feature])/population_dict[row['GEO_ID']]
    except:
        # print(i)
        # print('row[black_num_feature]: ', row[black_num_feature])
        # print('population_dict[row[GEO_ID]]: ', population_dict[row['GEO_ID']])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_black'] = 'None' # Both value are zeros

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909
